<a href="https://colab.research.google.com/github/tavares51/agro/blob/main/analise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Criação da Base de Dados

In [ ]:
vendedores = {1: 'João', 2: 'Maria', 3: 'Pedro', 4: 'Ana', 5: 'Carlos'}
# valores considerando a uma Saca (60kg)
produtos = [
    {'ProdutoId':1, 'Produto':'Soja', 'Valor': 136.42},
    {'ProdutoId':2, 'Produto':'Milho', 'Valor': 57.71},
    {'ProdutoId':3, 'Produto':'Sorgo', 'Valor': 38.00}
    ]
regioes = {1:'Norte', 2:'Sul', 3:'Leste', 4:'Oeste', 5:'Centro'}

In [ ]:
vendedores_df = pd.DataFrame(list(vendedores.items()), columns=['VendedorId', 'Nome'])
produtos_df = pd.DataFrame(produtos)
regioes_df = pd.DataFrame(list(regioes.items()), columns=['RegiaoId', 'Regiao'])

In [ ]:
clientes = list()
for i in range(1,16):
    clientes.append({'ClienteId':i, 'Nome':f'Cliente {i}', 'VendedorId':(i % 5) + 1})
clientes_df = pd.DataFrame(clientes)

In [ ]:
vendedores_df.head()

,VendedorId,Nome
0,1,João
1,2,Maria
2,3,Pedro
3,4,Ana
4,5,Carlos


In [ ]:
produtos_df.head()

,ProdutoId,Produto,Valor
0,1,Soja,136.42
1,2,Milho,57.71
2,3,Sorgo,38.00


In [ ]:
regioes_df.head()

,RegiaoId,Regiao
0,1,Norte
1,2,Sul
2,3,Leste
3,4,Oeste
4,5,Centro


In [ ]:
clientes_df.head()

,ClienteId,Nome,VendedorId
0,1,Cliente 1,2
1,2,Cliente 2,3
2,3,Cliente 3,4
3,4,Cliente 4,5
4,5,Cliente 5,1


Geração dos Dados de Venda

In [ ]:
SEED = 42
random.seed(SEED)

data_inicial = datetime(2023, 1, 1)
data_final = datetime(2023, 12, 31)
quantidades = list(range(50, 200))
num_registros = 200

vendas = []
for _ in range(num_registros):
    data_venda = data_inicial + timedelta(days=random.randint(0, (data_final - data_inicial).days))
    cliente_id = random.randint(clientes_df.ClienteId.min(), clientes_df.ClienteId.max())
    vendedor_id = clientes_df.loc[clientes_df['ClienteId'] == cliente_id, 'VendedorId'].values[0]
    produto_id = random.choice([1, 2, 3])
    regiao_id = random.randint(1, 5)
    quantidade = random.choice(quantidades)
    vendas.append([cliente_id, vendedor_id, produto_id, data_venda, regiao_id, quantidade])

colunas = ["ClienteId", "VendedorId", "ProdutoId", "DataVenda", "RegiaoId", "Quantidade"]
df_vendas = pd.DataFrame(vendas, columns=colunas)

In [ ]:
df_vendas.head()

,ClienteId,VendedorId,ProdutoId,DataVenda,RegiaoId,Quantidade
0,2,3,1,2023-11-24,3,112
1,3,4,3,2023-04-25,1,189
2,10,1,2,2023-02-14,1,57
3,4,5,1,2023-02-17,5,56
4,4,5,3,2023-10-15,5,157


In [ ]:
df_vendas['Valor_Venda'] = df_vendas.apply(lambda row: row['Quantidade'] * produtos_df[produtos_df['ProdutoId'] == row['ProdutoId']]['Valor'].values[0], axis=1)

In [ ]:
df_vendas.head()

,ClienteId,VendedorId,ProdutoId,DataVenda,RegiaoId,Quantidade,Valor_Venda
0,2,3,1,2023-11-24,3,112,15279.04
1,3,4,3,2023-04-25,1,189,7182.00
2,10,1,2,2023-02-14,1,57,3289.47
3,4,5,1,2023-02-17,5,56,7639.52
4,4,5,3,2023-10-15,5,157,5966.00


# Vizulização de Dados

In [ ]:
import plotly.express as px

In [ ]:
df_regiao_clientes = df_vendas.groupby('RegiaoId')['ClienteId'].value_counts().reset_index(name='QuantidadeClientes')
df_regiao_clientes = df_regiao_clientes.merge(regioes_df, on='RegiaoId')
quantidade_cliente_regiao = df_regiao_clientes.Regiao.value_counts()

In [ ]:
ax1 = px.bar(
    x=quantidade_cliente_regiao.index
    , y=quantidade_cliente_regiao.values
    , title='Quantidade de Clientes por Região'
    , labels={'x':'Região', 'y':'Quantidade de Clientes'}
    , color=quantidade_cliente_regiao.index
    , color_discrete_sequence=['#1f77b4', '#ffffff', '#2ca02c', '#d62728', '#9467bd']
    , width=550
    , height=350
    , text=quantidade_cliente_regiao.values
    , text_auto=True
    , template='plotly_dark'
    , opacity=0.7
)
ax1.update_layout(title_x=0.5)
ax1.show()

In [ ]:
import calendar

df_vendas_mes = df_vendas.copy()
df_vendas_mes['mes'] = df_vendas_mes['DataVenda'].dt.month
df_vendas_mes['ano'] = df_vendas_mes['DataVenda'].dt.year
df_vendas_mes = df_vendas_mes.groupby(['VendedorId', 'mes', 'ano'], as_index=False)['Valor_Venda'].sum()
#mes_filtro = int(input('Insira o mês para filtrar (1-12): '))
mes_filtro = 6
df_vendas_mes_filtrado = df_vendas_mes[df_vendas_mes['mes'] == mes_filtro]
df_vendas_mes_filtrado = df_vendas_mes_filtrado.groupby('VendedorId')['Valor_Venda'].sum().reset_index()
df_vendas_mes_filtrado = df_vendas_mes_filtrado.merge(vendedores_df, on='VendedorId')
ax2 = px.bar(
    df_vendas_mes_filtrado
    , x='Nome'
    , y='Valor_Venda'
    , color='Nome'
    , color_discrete_sequence=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    , title='Valor Total de Vendas por Vendedor - Mês: ' + str(calendar.month_name[mes_filtro])
    , labels={'x':'Vendedor', 'y':'Valor Total de Vendas'}
    , width=550
    , height=350
    , text='Valor_Venda'
    , text_auto=True
    , template='plotly_dark'
    , opacity=0.7
    )
ax2.update_layout(title_x=0.5)
ax2.show()
